# ASPerf: Perfect hasing with ASP


In [4]:
!python2 asperf.py -h

usage: asperf.py [-h] [--template TEMPLATE] [--yosys] [-b B] [-n N] [-p P]
                 [keyfile]

Generate a perfect hash function.

positional arguments:
  keyfile              a newline delimited array of keys to be hashed

optional arguments:
  -h, --help           show this help message and exit
  --template TEMPLATE  a function template lp file for asperf to fill in
  --yosys              generate a yosys circuit for additional operations
  -b B                 number of bits per key
  -n N                 number of operations
  -p P                 parallel mode


By default the utility uses a simplified hash function generator which is only allowed to use xor,and,not,and shr (mod n). Although this is powerful enough for many instances, problems such as the states hashing one which we are about to tackle prove very difficult to find good solutions for using these operations. In order to support more complex operations we updated the yosys code from P2 and used this to implement mod, mul, and sub of for arbitrary numbers of bits.

# Basic ASP formulation

These basic rules define some global properties about the network and specify how to nondeterministically build it.

In [8]:
!cat circuits/globals.lp


bit(0;1).

#const bits=16.
#const nodes=5.

bits(bits).

bit_index(0..bits-1).

id(0..nodes-1).

rooted(0).
rooted(B) :- rooted(A), edge(A,B).
node(I) :- rooted(I).

In [7]:
!cat circuits/generate.lp


%%% Each node has 0-2 children and is connected to the root
%%% Childen have greater ids than their parents
0 {edge(A,B):id(B),B>A} 2 :- rooted(A).
    
%%%Choose one operation for each node

1{op(I, T):bin_op(T)}1 :- binary(I).
1{op(I, T):un_op(T)}1 :- unary(I).
1{op(I, T):null_op(T)}1 :- leaf(I).
    

%%%Create constants
1 {const_bit(I,B,0..1)} 1 :- op(I,const),bit_index(B).



# Verification and optimization

Most of the verification and optimization code is in verify.lp, however we also include circuit_int.lp and circuit_lib.lp, which include the interpretation and representation code for yosys-generated circuits, respectively.

Note also that the code for printing out the C functon is at the bottom of the verify.lp file and is called whenever a new model is generated.

In [9]:
!cat circuits/circuit_int.lp

input_port(Port) :- device_port_direction(Device,Port,input,C).

output_port(Port) :- device_port_direction(Device,Port,output,C).

port_width(Port,Size) :-
    device_port_width(Device,Port,Size).

% Define all the gate rules
signal(I,K,Device,Out,1-(1-V1)*(1-V2)) :-
    op(I,Device),
    device_gate(Device,or_gate,(In1,In2,Out)),
    signal(I,K,Device,In1,V1); signal(I,K,Device,In2,V2).

signal(I,K,Device,Out,V1*V2) :-
    op(I,Device),
    device_gate(Device,and_gate,(In1,In2,Out)),
    signal(I,K,Device,In1,V1), signal(I,K,Device,In2,V2).

signal(I,K,Device,Out,1-V1) :-
    op(I,Device),
    device_gate(Device,not_gate,(In,Out)),
    signal(I,K,Device,In,V1).

% get input signal
signal(I,K,Device,Wire,Value) :-
    op(I,Device),
    input_signal(I,K,Port,Bit,Value),
    device_port_bit_wire(Device,Port,Bit,Wire).

% get output signal
output_signal(I,K,Bit,Value) :-
    signal(I,K,Device,Wire,Value),
    output_port(Port),
    device_port_bit_wire(Device,Port,Bit,Wire).

output_sign

In [10]:
!head circuits/circuit_lib.lp

device(mul).
device_port_direction(mul,mulin1,input,0).
device_port_width(mul,mulin1,16).
device_port_bit_wire(mul,mulin1,0,2).
device_port_bit_wire(mul,mulin1,1,3).
device_port_bit_wire(mul,mulin1,2,4).
device_port_bit_wire(mul,mulin1,3,5).
device_port_bit_wire(mul,mulin1,4,6).
device_port_bit_wire(mul,mulin1,5,7).
device_port_bit_wire(mul,mulin1,6,8).


In [11]:
!cat circuits/verify.lp


binary(I) :- node(I), 2{edge(I,C)}2.
unary(I) :- node(I), 1{edge(I,C)}1.
leaf(I) :- node(I), 0{edge(I,C)}0.
      
bin_op(xor;and;mod;mul).
un_op(neg).
null_op(const;key).

type(T) :- bin_op(T).
type(T) :- un_op(T).
type(T) :- null_op(T).
    
%%%Evaluate the dag, (Key, Id, Bit, Value)
%%% Basic operations (and,xor,neg,const,key)

val_bit(K,I,B,V1&V2) :- 
    op(I,and),
    edge(I,I1),
    edge(I,I2),
    I1 < I2, 
    val_bit(K,I1,B,V1), 
    val_bit(K,I2,B,V2).
    
val_bit(K,I,B,V1^V2) :- 
    op(I,xor),
    edge(I,I1),
    edge(I,I2),
    I1 < I2, 
    val_bit(K,I1,B,V1), 
    val_bit(K,I2,B,V2).
    
val_bit(K,I,B,1-V1) :- 
    op(I,neg),
    edge(I,I1),
    val_bit(K,I1,B,V1).
    
val_bit(K,I,B,V) :- 
    key(K), 
    op(I,const), 
    const_bit(I,B,V).
    
val_bit(K,I,B,V) :- 
    key_bit(K,B,V), 
    op(I,key).
    
%%% Advanced operations (mul,shl,shr,mod,div,add,sub), need yosys circuits
input_signal(I,K,P,B,V1) :- 
    op(I,D),
    unary(D),
    edge(I,I1),
    val_bit(K,

# Test instance: 50 states + 9 territories

We found that some of the most difficult things for our program to hash were lists of numbers that were fairly clusterred together, but not clusterred together enough that they were anywhere near a perfect hash when just applying some offsets. We believe that this has to do with how many bits of the numbers need to be examined in order to disambiguate them when trying to fit them into a smaller range. 

This "50 states" instance is obtained by taking the two letter codes for each state (or territory) and concatenating their values together. For example CA = 43 41 (hex) = 17217 (dec) in the list below.

This instance seems to have the clustering property described above, and is this difficult for our program to hash efficient. In the end, we obtain a range approximately 4 times the number of keys.


In [50]:
!cat states

16715
16716
16722
16723
16730 
17217 
17231
17236 
17475
17477 
17996 
17997 
18241 
18261 
18505 
18753
18756 
18764
18766 
19283 
19289
19521 
19777
19780 
19781 
19784 
19785
19790 
19791 
19792 
19795
19796
20035 
20036 
20037 
20040
20042
20045 
20054 
20057 
20296
20299 
20306 
20545 
20562 
20567
21065
21315 
21316 
21582
21592
21844
22081 
22089
22100
22337 
22345
22358 
22361


# Results

In order to save room in this notebook we copied the result of an earlier run into the states_run_log file and simply show the head and tail of that log below. 

To get similar results to these, simply run

./asperf.py states --yosys --template template.lp

Where states is the file shown above containing the 16-bit representation of each state and template.lp specifies a hash function of the form Ax mod B for some constants A and B (and an appropriate choice rule to choose those constants), as shown below.

In [23]:
!cat template.lp

1 {const_bit(I,B,0..1)} 1 :- op(I,const),bit_index(B).
  
op(0, mod).

edge(0,1).
edge(0,2).

op(2, const).
op(1,mul).

edge(1,3).
edge(1,4).

op(3, key).
op(4, const).


In [12]:
!head states_run_log

clingo version 5.2.0
Reading from keys_states.lp ...
circuit_int.lp:49:5-51: info: atom does not occur in any rule head:
  device_port_bit_literal(Device,Port,Bit,Value)

Solving...
int hash(int key) {
    int i2 = 32764;
    int i4 = 7222;
    int i3 = key;


In [22]:
!tail -n 70 states_run_log

int hash(int key) {
    int i2 = 215;
    int i4 = 42351;
    int i3 = key;
    int i1 = 0xffff & (i3 * i4);
    int i0 = i1 % i2;
    return i0;
}
hash(16715) = 59
hash(16716) = 94
hash(16722) = 11
hash(16723) = 7
hash(16730) = 174
hash(17217) = 191
hash(17231) = 56
hash(17236) = 153
hash(17475) = 43
hash(17477) = 113
hash(17996) = 137
hash(17997) = 172
hash(18241) = 159
hash(18261) = 156
hash(18505) = 182
hash(18753) = 55
hash(18756) = 121
hash(18764) = 69
hash(18766) = 139
hash(19283) = 132
hash(19289) = 49
hash(19521) = 202
hash(19777) = 62
hash(19780) = 128
hash(19781) = 124
hash(19784) = 190
hash(19785) = 10
hash(19790) = 107
hash(19791) = 142
hash(19792) = 177
hash(19795) = 28
hash(19796) = 24
hash(20035) = 168
hash(20036) = 164
hash(20037) = 199
hash(20040) = 50
hash(20042) = 81
hash(20045) = 147
hash(20054) = 130
hash(20057) = 196
hash(20296) = 86
hash(20299) = 152
hash(20306) = 104
hash(20545) = 209
hash(20562) = 140
hash(20567) = 22
hash(21065) = 53
hash(21315) = 211
hash(21

# Verification

In order to verify our hash function was in fact valid we copied it into a small C++ program

In [27]:
!cat states.cpp

#include <iostream>
using namespace std;

int phash(int key) {
  int i2 = 215;
  int i4 = 42351;
  int i3 = key;
  int i1 = 0xFFFF & (i3 * i4);
  int i0 = i1 % i2;
  return i0;
}


int main() {

  std::cout << "phash(16715) = " << phash(16715) << std::endl;
  std::cout << "phash(16716) = " << phash(16716) << std::endl;
  std::cout << "phash(16722) = " << phash(16722) << std::endl;
  std::cout << "phash(16723) = " << phash(16723) << std::endl;
  std::cout << "phash(16730) = " << phash(16730) << std::endl;
  std::cout << "phash(17217) = " << phash(17217) << std::endl;
  std::cout << "phash(17231) = " << phash(17231) << std::endl;
  std::cout << "phash(17236) = " << phash(17236) << std::endl;
  std::cout << "phash(17475) = " << phash(17475) << std::endl;
  std::cout << "phash(17477) = " << phash(17477) << std::endl;
  std::cout << "phash(17996) = " << phash(17996) << std::endl;
  std::cout << "phash(17997) = " << phash(17997) << std::endl;
  std::cout << "phash(18241) = " << phash(18241) 

In [28]:
!g++ states.cpp -o states && ./states

phash(16715) = 59
phash(16716) = 94
phash(16722) = 11
phash(16723) = 7
phash(16730) = 174
phash(17217) = 191
phash(17231) = 56
phash(17236) = 153
phash(17475) = 43
phash(17477) = 113
phash(17996) = 137
phash(17997) = 172
phash(18241) = 159
phash(18261) = 156
phash(18505) = 182
phash(18753) = 55
phash(18756) = 121
phash(18764) = 69
phash(18766) = 139
phash(19283) = 132
phash(19289) = 49
phash(19521) = 202
phash(19777) = 62
phash(19780) = 128
phash(19781) = 124
phash(19784) = 190
phash(19785) = 10
phash(19790) = 107
phash(19791) = 142
phash(19792) = 177
phash(19795) = 28
phash(19796) = 24
phash(20035) = 168
phash(20036) = 164
phash(20037) = 199
phash(20040) = 50
phash(20042) = 81
phash(20045) = 147
phash(20054) = 130
phash(20057) = 196
phash(20296) = 86
phash(20299) = 152
phash(20306) = 104
phash(20545) = 209
phash(20562) = 140
phash(20567) = 22
phash(21065) = 53
phash(21315) = 211
phash(21316) = 207
phash(21582) = 46
phash(21592) = 64
phash(21844) = 38
phash(22081) = 112
phash(22089) = 

This is the same output as in our ASP implementation, so our function works!

It should be noted that when we hash a collection of "hard to hash numbers" their result is also relatively hard to hash, as demonstrated below by trying to hash the hash values produced above. This further emphasizes out point that the hardness of hashing a collection of numbers is largely dependent on the number of bits that need to be examined to disambiguate two numbers.

In [36]:
!./statea | head

59
94
11
7
174
191
56
153
43
113


In [31]:
!./statea > stateatest

Running !python2 asperf.py stateatest --yosys -n 4 -p 4 we obtained

In [49]:
!tail -n 68 hash_again

int phash(int key) {
    int i3 = 182;
    int i2 = key;
    int i0 = i2 % i3;
    return i0;
}
hash(94) = 94
hash(174) = 174
hash(56) = 56
hash(172) = 172
hash(156) = 156
hash(182) = 0
hash(132) = 132
hash(202) = 20
hash(62) = 62
hash(128) = 128
hash(124) = 124
hash(190) = 8
hash(10) = 10
hash(142) = 142
hash(28) = 28
hash(24) = 24
hash(168) = 168
hash(164) = 164
hash(50) = 50
hash(130) = 130
hash(196) = 14
hash(86) = 86
hash(152) = 152
hash(104) = 104
hash(140) = 140
hash(22) = 22
hash(46) = 46
hash(64) = 64
hash(38) = 38
hash(112) = 112
hash(60) = 60
hash(74) = 74
hash(148) = 148
hash(96) = 96
hash(180) = 180
hash(59) = 59
hash(11) = 11
hash(7) = 7
hash(191) = 9
hash(153) = 153
hash(43) = 43
hash(113) = 113
hash(137) = 137
hash(159) = 159
hash(55) = 55
hash(121) = 121
hash(69) = 69
hash(139) = 139
hash(49) = 49
hash(107) = 107
hash(177) = 177
hash(199) = 17
hash(81) = 81
hash(147) = 147
hash(209) = 27
hash(53) = 53
hash(211) = 29
hash(207) = 25
hash(31) = 31
Answer: 10
edge(0,3) edg